# CONVERSIÓN A HORARIOS

En este *notebook* se convierten los datos originales de *Bicing*. El archivo original registra la cantidad de bicicletas en cada estación cada 5 minutos (aproximadamente), por lo que cada hora contiene aproximadamente unos 12 registros. Para reducir la cantidad de información, se calcula el promedio de los registros dentro de cada hora.

In [1]:
import pandas as pd

In [2]:
datos = pd.read_parquet('../../1-DATOS/2-DATOS PROCESADOS/BICING/INFORMACION COMPLETA/BICICLETAS.parquet')

In [3]:
def procesado(datos, fecha_inicial, fecha_final):
    # Resamplear los datos por hora y calcular el promedio
    datos_horarios = datos.resample('h').mean()

    # Mantener los nombres originales de las columnas
    datos_horarios.index.name = 'FECHA'

    # Filtrar por rango de fechas
    datos_horarios = datos_horarios[(datos_horarios.index >= fecha_inicial) & (datos_horarios.index < fecha_final)] 

    return datos_horarios

Para desarrollar estos modelos de predicción y planificación, se ha decidido utilizar los datos de los años 2022 y 2023. Aunque existen registros desde 2019, se ha limitado el periodo de análisis para evitar que el impacto de la pandemia de COVID-19 sesgue los patrones de demanda y uso de bicicletas. Asimismo, se ha decidido no incluir el año 2024, ya que el proyecto comenzó en septiembre de dicho año.

Finalmente, se han descartado aquellas series temporales que contienen más de 1500 horas sin datos durante el período analizado. Esto equivale  a un poco más de dos meses sin información registrada (que no tienen que ser dos meses continuos, sino que pueden estar distribuidos en distintas fechas).

In [4]:
datos_horarios_completos = procesado(datos, '2018', '2025')
datos_horarios_trabajo = procesado(datos, '2022', '2024')
datos_horarios_filtrado = datos_horarios_trabajo.loc[:, datos_horarios_trabajo.isna().sum() <= 1500]

In [5]:
datos_horarios_filtrado.head(5)

station_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0,...,450.0,452.0,458.0,490.0,465.0,502.0,503.0,52.0,91.0,467.0
FECHA,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,14.636364,15.181818,5.454545,3.818182,5.000000,11.272727,7.545455,16.000000,9.090909,11.000000,...,10.000000,15.272727,6.0,14.0,13.909091,1.818182,9.090909,27.545455,33.090909,0.000000
2022-01-01 01:00:00,12.750000,16.000000,8.250000,4.000000,5.166667,10.583333,9.000000,17.666667,7.833333,11.000000,...,10.000000,15.750000,6.0,14.0,12.750000,3.000000,10.000000,26.833333,33.000000,0.000000
2022-01-01 02:00:00,12.076923,16.384615,9.538462,3.692308,5.000000,8.692308,10.000000,19.461538,6.615385,10.615385,...,10.000000,15.000000,6.0,14.0,11.538462,4.000000,10.000000,28.230769,30.384615,0.923077
2022-01-01 03:00:00,13.000000,16.083333,9.416667,2.000000,5.000000,8.000000,9.083333,22.250000,11.000000,10.000000,...,9.083333,14.666667,6.0,14.0,11.083333,4.416667,10.000000,24.000000,30.000000,1.000000
2022-01-01 04:00:00,13.000000,16.000000,9.000000,2.000000,5.000000,8.000000,9.000000,23.000000,11.750000,10.000000,...,10.666667,14.000000,6.0,14.0,11.333333,5.000000,10.000000,18.666667,30.000000,1.833333


In [6]:
datos_horarios_completos.to_parquet('../../1-DATOS/2-DATOS PROCESADOS/BICING/INFORMACION COMPLETA/BICICLETAS_HORARIO_COMPLETO.parquet')
datos_horarios_trabajo.to_parquet('../../1-DATOS/2-DATOS PROCESADOS/BICING/INFORMACION COMPLETA/BICICLETAS_HORARIO_2022_2023.parquet')
datos_horarios_filtrado.to_parquet('../../1-DATOS/2-DATOS PROCESADOS/BICING/INFORMACION COMPLETA/BICICLETAS_HORARIO_2022_2023_FILTRADO.parquet')